# Fetch Data

In [1]:
import pandas as pd
import json

In [2]:
def clean_name_1(name):
    
    name = name.split(' ')
    
    if len(name) <= 1:
        return name[0].lower()
    else:
        
        return '-'.join(name).lower()


In [3]:
def clean_name_2(name):
    
    name = name.split(' ')
    
    if len(name) <= 1:
        return name[0].lower()
    else:
        
        return '-'.join(name[:-1]).lower()


In [4]:
def clean_genres_1(genre):
    return ','.join(list(json.loads(genre).values())).lower()

    

In [5]:
def clean_genres_2(genre):
    return ','.join(genre.split('|')).lower()

In [6]:
def fetch_data():

    columns=[
        'movieId',
        'freebase_id',
        'title',
        'release-date',
        'box-office-revenue',
        'runtime',
        'languages',
        'countries',
        'genres',
        ]

    drop_columns = [
        'freebase_id',
        'release-date',
        'box-office-revenue',
        'runtime',
        'languages',
        'countries',
        ]

    df1 = pd.read_csv('data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None, names=columns).drop(columns=drop_columns)
    df2 = pd.read_csv('data/ml-latest-small/movies.csv')
    
    df1['title'] = df1['title'].apply(clean_name_1)
    df2['title'] = df2['title'].apply(clean_name_2)
    df1['genres'] = df1['genres'].apply(clean_genres_1)
    df2['genres'] = df2['genres'].apply(clean_genres_2)
    
    return df1, df2

In [7]:
df1, df2 = fetch_data()

In [14]:
df1

,movieId,title,genres
0,975900,ghosts-of-mars,"thriller,science fiction,horror,adventure,supe..."
1,3196793,getting-away-with-murder:-the-jonbenét-ramsey-...,"mystery,biographical film,drama,crime drama"
2,28463795,brun-bitter,"crime fiction,drama"
3,9363483,white-of-the-eye,"thriller,erotic thriller,psychological thriller"
4,261236,a-woman-in-flames,drama
...,...,...,...
81736,35228177,mermaids:-the-body-found,drama
81737,34980460,knuckle,"biographical film,drama,documentary"
81738,9971909,another-nice-mess,"satire,comedy"
81739,913762,the-super-dimension-fortress-macross-ii:-lover...,"science fiction,japanese movies,adventure,anim..."


In [8]:
from tqdm.notebook import tqdm

In [9]:
df1_titles = df1['title'].tolist()

In [10]:
df2_titles = df2['title'].tolist()

In [11]:
joined = [i for i in tqdm(df1) if i in df2]

  0%|          | 0/81741 [00:00<?, ?it/s]

In [68]:
df2

,movieId,title,genres
0,1,toy-story,"adventure,animation,children,comedy,fantasy"
1,2,jumanji,"adventure,children,fantasy"
2,3,grumpier-old-men,"comedy,romance"
3,4,waiting-to-exhale,"comedy,drama,romance"
4,5,father-of-the-bride-part-ii,comedy
...,...,...,...
9737,193581,black-butler:-book-of-the-atlantic,"action,animation,comedy,fantasy"
9738,193583,no-game-no-life:-zero,"animation,comedy,fantasy"
9739,193585,flint,drama
9740,193587,bungo-stray-dogs:-dead-apple,"action,animation"


In [93]:
def fetch_concurrant(df1, df2):
    
    empty_df = {'df1_id' : [], 'df2_id' : [],
                'title'  : [], 'genres' : []}
    titles = df1['title'].tolist()
    for i in tqdm(range(len(df2))):
        
        if df2.loc[i]['title'] in titles:
            empty_df['df1_id'].append(df1.loc[i]['movieId'])
            empty_df['df2_id'].append(df2.loc[i]['movieId'])
            empty_df['title'].append(df2.loc[i]['title'])
            empty_df['genres'].append(df1.loc[i]['genres']+df2.loc[i]['genres'])
    
    return empty_df
        

In [94]:
conc = fetch_concurrant(df1, df2)

  0%|          | 0/9742 [00:00<?, ?it/s]

In [167]:
df = pd.DataFrame(conc)

In [168]:
df.sort_values('title')

,df1_id,df2_id,title,genres
3723,28417619,27751,'salem's-lot,"crime fictiondrama,horror,mystery,thriller"
430,1941497,779,'til-there-was-you,"science fiction,adventure,satire,fantasy,comed..."
4574,31028212,69757,(500)-days-of-summer,"crime fiction,thriller,dramacomedy,drama,romance"
3404,6259249,8169,*batteries-not-included,"computer animationchildren,comedy,fantasy,sci-fi"
1708,29288150,3420,...and-justice-for-all,"romance film,comedy filmdrama,thriller"
...,...,...,...,...
4063,23691133,47384,zoom,"short filmadventure,comedy,drama,fantasy"
5485,28837893,156553,zoom,"romantic comedyanimation,comedy,drama"
5284,2170326,116207,zulu,"lgbt,gay,gay interest,drama,docudrama,gay them..."
2774,22911789,5899,zulu,"indie,documentaryaction,drama,war"


In [81]:
df.to_csv('data/two_dataset.csv', index=False)

In [100]:
with open('data/MovieSummaries/plot_summaries.txt', 'r') as file:
    plot_sums = file.read().split('\n')

In [109]:
dic = {}
for i in plot_sums:
    x = i.split('\t')
    
    try:
        dic[x[0]] = x[1]
    except:
        print(x)

['']


In [174]:
df.iloc[4]

df1_id                                             18998739
df2_id                                                    7
title                                               sabrina
genres    family film,fantasy,adventure,world cinemacome...
Name: 6, dtype: object

In [176]:
for i in range(len(df)):
    try:
        if str(df.iloc[i]['df1_id']) in dic.keys():
            df = df.drop([df.index[i]])
    except:
        

        
        
        
        
       

IndexError: single positional indexer is out-of-bounds

In [ ]:
df.sort_values('title')

In [156]:
if 2170326 not in dic.keys():
    print('poop')

poop
